In [3]:
import pandas as pd 
import functools
import numpy as np 

In [4]:
file = 'test.xlsx'

In [5]:
def to_camelcase(string):
    output = ''.join(x for x in string.title() if x.isalnum())
    return output[0].lower() + output[1:]
def capitalize_seh(string):
    return string.replace('Seh', 'SEH') 

In [6]:
def get_cols_as_dict(key_col, value_col, start_index, end_index=None, 
                     datetime_index=False, datetime_fmt=''):
    
    nrows = end_index-(start_index-1) if end_index is not None else 1
    df = pd.read_excel(file, skiprows=start_index-1,  nrows=nrows, 
                       usecols = f'{key_col},{value_col}', header=None, index_col=0)
    
    series = df.iloc[:, 0].infer_objects()
    dct = series.to_dict()
    
    mappings = [str, to_camelcase, capitalize_seh]
    for mapping in mappings:
        dct = {mapping(k):v for k,v in dct.items()}
        
    return dct

In [34]:
root = {
    **get_cols_as_dict('B', 'D', 13),  # 1
    **get_cols_as_dict('B', 'D', 15),  # 2 
    **get_cols_as_dict('B', 'D', 17, 18),  # 3a
    '3b': get_cols_as_dict('C', 'D', 20, 31),  # 3b
    '3c': get_cols_as_dict('C', 'D', 34, 40),  # 3c
    '3d': get_cols_as_dict('B', 'D', 42, 43),  # 3d
    '3e': get_cols_as_dict('B', 'D', 45, 49),  # 3e
    'stapeldrukte': {  # 3f
        'mon': {'max': get_cols_as_dict('C', 'D', 52, 75), 'gem': get_cols_as_dict('C', 'E', 52, 75)},
        'tue': {'max': get_cols_as_dict('C', 'D', 77, 100), 'gem': get_cols_as_dict('C', 'E', 77, 100)},
        'wed': {'max': get_cols_as_dict('C', 'D', 102, 125), 'gem': get_cols_as_dict('C', 'E', 102, 125)},
        'thu': {'max': get_cols_as_dict('C', 'D', 127, 150), 'gem': get_cols_as_dict('C', 'E', 127, 150)},
        'fri': {'max': get_cols_as_dict('C', 'D', 152, 175), 'gem': get_cols_as_dict('C', 'E', 152, 175)},
        'sat': {'max': get_cols_as_dict('C', 'D', 177, 200), 'gem': get_cols_as_dict('C', 'E', 177, 200)},
        'sun': {'max': get_cols_as_dict('C', 'D', 202, 225), 'gem': get_cols_as_dict('C', 'E', 202, 225)},
        **get_cols_as_dict('B', 'D', 226)  # unpack to key and value
    },
    '3g': get_cols_as_dict('B', 'D', 228, 230),
    'specialisme': get_cols_as_dict('C', 'D', 233, 242),  # 4a
    '4b': get_cols_as_dict('B', 'D', 244, 247),
    '4c': get_cols_as_dict('B', 'D', 250, 253),
    '5': '',
    '6a': get_cols_as_dict('B', 'D', 259),
    '6b': get_cols_as_dict('B', 'D', 261),
    '6c': get_cols_as_dict('B', 'D', 263),
    '6d': get_cols_as_dict('B', 'D', 265),
    '6e': get_cols_as_dict('B', 'D', 267)
}

week_keys = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']

for col in week_keys:
    root['stapeldrukte'][col]['max'] = {f'H{k[0:2]}': v for k,v in root['stapeldrukte'][col]['max'].items()}
    root['stapeldrukte'][col]['gem'] = {f'H{k[0:2]}': v for k,v in root['stapeldrukte'][col]['gem'].items()}

times = list(root['stapeldrukte']['mon']['max'].keys())
max_week = np.max([list(root['stapeldrukte'][col]['max'].values()) for col in week_keys], axis=0).astype(float)
gem_week = np.mean([list(root['stapeldrukte'][col]['gem'].values()) for col in week_keys], axis=0).astype(float)

root['stapeldrukte']['week'] = {'max': dict(zip(times, max_week)), 'gem': dict(zip(times, gem_week))}

In [35]:
import json
with open('seh.json', 'w') as filepath:
    json.dump(root, filepath)